### Load the most recent events

In [26]:
import pandas as pd

In [27]:
sepEvents = pd.read_pickle("Data/SepEvents_50K_Sep25.pkl")
octEvents = pd.read_pickle("Data/OctEvents_50K_Sep26.pkl")
novEvents = pd.read_pickle("Data/NovEvents_100K.pkl")
decEvents = pd.read_pickle("Data/DecEvents_100K.pkl")
janfebEvents = pd.read_pickle("Data/JanFebEvents_100K.pkl")
maraprEvents = pd.read_pickle("Data/MarAprEvents_100K.pkl")

In [3]:
allEvents = pd.concat([sepEvents, octEvents, novEvents, decEvents, janfebEvents, maraprEvents])

In [4]:
from festivalTools import *

/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


### Split into chunks that are agreeable with the NewsRiver API. Then generate news articles

In [19]:
allEvents1 = allEvents.iloc[:224]
allEvents2 = allEvents.iloc[224:448]
allEvents3 = allEvents.iloc[448:600]
allEvents4 = allEvents.iloc[600:]

In [20]:
allNews1 = generateNewsDataFrame(allEvents1['Event Name'], allEvents1['Location'])

In [23]:
allNews2 = generateNewsDataFrame(allEvents2['Event Name'], allEvents2['Location'])

In [24]:
allNews3 = generateNewsDataFrame(allEvents3['Event Name'], allEvents3['Location'])

In [25]:
allNews4 = generateNewsDataFrame(allEvents4['Event Name'], allEvents4['Location'])
allNews = pd.concat([allNews1, allNews2, allNews3, allNews4])

### Merge news articles into a single text dataframe

In [6]:
allText = generateTextDataFrame(allNews, min_articles=2)

In [7]:
allText

,City,Event Name,Text
0,Bethlehem,Peanut Butter Festival,National Peanut Butter and Jelly Day is celebr...
1,Durban,Lesbian Film Festival,A lesbian film festival is returning to Paduca...
2,0 Springs 0 Blue Springs Name: City...,Blue Springs Fall Fun Festival,"Burgers are a classic KC menu item. ""As a stea..."
3,"0 Kasama 0 Naga Name: City, dtype: object",Peñafrancia Festival,Naga City in Camarines Sur is also known as th...
4,0 Cancún 0 Santa Clara 0 Lafay...,Wine Festival,Hours before guests grooved to iconic 1990s tu...
5,Bintulu,Borneo International Kite Festival,Click to share on WhatsApp (Opens in new windo...
6,Kuching,Kuching Waterfront Jazz Festival,Click to share on WhatsApp (Opens in new windo...
7,"0 Auckland 0 Atlanta Name: City, dtype:...",Peace Festival,Ahead of the forthcoming African Youth Festiva...
8,0 Angeles 0 Los Angeles 0 Los Ang...,LA Film Festival,Fresh off its ingestion of lip syncing app Mus...
9,"0 Kazan 0 Cairo Name: City, dtype: object",Kazan International Festival,"International-award winning film Halda, direct..."


In [29]:
allNews.to_pickle("Data/allNewsSep26.pkl")
allText.to_pickle("Data/allTextSep26.pkl")
allEvents.to_pickle("Data/allEventsSep26.pkl")

### Merge with allText from yesterday

In [8]:
allTextPr = pd.read_pickle("Data/allTextSep23.pkl")

In [9]:
allNewsPr = pd.read_pickle("Data/allNewsSep23.pkl")

### Edit allText to deal with duplications, etc.

In [10]:
#determine the correct city by searching for how many times it appears in text
def determineCityName(cities, text):
    if len(cities) == 1:
        return cities[0]
    else:
        counts = []
        for city in cities:
            counts.append(text.count(city.lower()))
        max_index = counts.index(max(counts))
        return cities[max_index]

In [11]:
for index, (cities, text) in enumerate(zip(allTextPr['City'], allTextPr['Text'])):
    if type(cities) != str:
        allTextPr.iloc[index]['City'] = determineCityName(cities.values, text)
        
for index, (cities, text) in enumerate(zip(allText['City'], allText['Text'])):
    if type(cities) != str:
        allText.iloc[index]['City'] = determineCityName(cities.values, text)
        

In [12]:
allTextCom = pd.concat([allText, allTextPr])
allTextCom.drop_duplicates(subset=['Event Name', 'City'], inplace=True, keep='first')

In [13]:
allTextCom

,City,Event Name,Text
0,Bethlehem,Peanut Butter Festival,National Peanut Butter and Jelly Day is celebr...
1,Durban,Lesbian Film Festival,A lesbian film festival is returning to Paduca...
2,Springs,Blue Springs Fall Fun Festival,"Burgers are a classic KC menu item. ""As a stea..."
3,Kasama,Peñafrancia Festival,Naga City in Camarines Sur is also known as th...
4,Cancún,Wine Festival,Hours before guests grooved to iconic 1990s tu...
5,Bintulu,Borneo International Kite Festival,Click to share on WhatsApp (Opens in new windo...
6,Kuching,Kuching Waterfront Jazz Festival,Click to share on WhatsApp (Opens in new windo...
7,Auckland,Peace Festival,Ahead of the forthcoming African Youth Festiva...
8,Angeles,LA Film Festival,Fresh off its ingestion of lip syncing app Mus...
9,Kazan,Kazan International Festival,"International-award winning film Halda, direct..."


## Now we can add summaries

In [14]:
from gensim.summarization.summarizer import summarize
summaries = []
for event, city in zip(allText['Event Name'], allText['City']):
    eventNews = allNews[allNews['Event Name']==event]
    count = 0
    countInd = 0
    for index, news in enumerate(eventNews['Article Text']):
        cityCount = news.count(city)
        eventCount = news.count(event)
        if cityCount*eventCount > count:
            count = cityCount*eventCount
            countInd = index
            print(count)
    print(event + ", " + city)
    try:
        summary = summarize(eventNews.iloc[countInd]['Article Text'], word_count = 80)
        summaries.append(summary.replace("\n", " "))
    except:
        summaries.append("")
        print("Error Summerizing!")
    print(summary)
    #print(eventNews.iloc[countInd]['Article Text'])
    print("\n")
    

Peanut Butter Festival, Bethlehem
Peanut Butter and Jelly sandwiches were thought to be invented during the American Civil war, when the men mixed their peanut ration and jelly ration to create a delicious new treat while out on the field.
According to the Peanut Board, during World War II, both peanut butter and jelly were part of the United States soldiers military ration list.
720 pounds of bread, 493 pounds of peanut butter and 129 pounds of jelly.


Lesbian Film Festival, Durban
A lesbian film festival is returning to Paducah for its third year this May. The organizers bill the event as “The only all-lesbian film festival in the U.S.” The Cinema Systers Film Festival includes screenings, question and answer sessions with filmmakers and a quilted prayer flag exhibit.
“I think that Cinema Systers Film Festival being dedicated to women filmmakers gives the public a view into another world from a different perspective--one that they’re not used to seeing on screen,” Petrie said.


1
B

From the Spring Racing Carnival to Melbourne Fashion Week and even our very own Oktoberfest, the city has a lot on offer.
Running throughout the first week in spring, the festival celebrates all things Australian and international fashion and retail.
Coming to Melbourne for the very first time, Lost Picnic promises a day jam-packed full of boho music, fun and food.
Back with its ninth instalment, Melbourne Music Week is a celebration of local and international talent.


72
78
Brisbane Festival, Brisbane
This free event is at Mundoolun Community Centre Park, Scarborough CRT, Mundoolun on Saturday, September 22 from 10am to 11am email:info@zoccersports.com.au Details: 0488 622 875 Fishing by the River (5-11 years) Learn the basics of shore-based fishing, knot tying, rod casting and heaps of other fishing tips to make your next adventure more successful.
The bus returns at 2pm Details: 3442 1818 INDOORS Learn to swim: 5-11 years (Sessions every 30 minutes) This school holiday program at G

4
The Festival, Edinburgh
Enabling more accessibility and inclusion our friends at the The Festival City Theatres Trust, which runs the Festival and King’s theatres, once again expand their programme of relaxed performances for audiences of all ages for the 2017/18 festive season.
“Ahead of an inclusive performance of Scottish Ballet’s The Nutcracker, the Festival City Theatres Trust is delighted to present its fullest ever programme of performances which embrace the relaxed approach, making performances accessible to audiences of all ages with mild to complex support needs.


4
12
Swansea International Festival, Swansea
The three-day event will take place on the weekend of October 5-7, closing the long-running Swansea International Festival.
“This year’s Swansea Fringe Festival will kick off with a special launch event in Unit 19 on October 4 with Hollie McNish.
Alternatively, you can visit www.theswanseafringe.com or www.swanseamusichub.co.uk Swansea Music Hub was recently involved i

To revive and introduce this experience, we at Shikara felt the need to introduce the Kashmiri Food Festival, to relish this cuisine, which is often described as platter from heaven.” The vegetarian menu at the food festival will have Palak Shorba with paneer, Nadur Monji, Dum Olav and Paneer tandoori as starters, Palak/Haaq saag, Ruwangan Chaman, Rajmah with shalgum, Nadur Yakhni, Chokh Wangun in the main course and Kong Phirni, Shufta and Kehwa in desserts.


Kite Festival, Kolkata
Kite festival in pictures: Hundreds turned up for the major event Established in 1980 and the largest of its kind in the world, the Dieppe International Kite Festival unites more than a thousand professionals from five continents on Normandy’s beautiful Alabaster Coast in France.
Given its international representation, the event is considered to be the biggest kite festival in the world.
Around forty countries are invited to the event and this year, Cambodia is the festival’s guest of honour, hosting a gra

When Netflix shut down production of “House of Cards,” the award-winning television show filmed in the Baltimore area, because of sexual harassment allegations made against the show’s star, Kevin Spacey, it made national news.
Ditto when Baltimore artist Paul Rucker’s art exhibit of life-sized mannequins clad in Ku Klux Klan robes was deemed so potentially inflammatory that a Pennsylvania college closed the show to the public.


Roots Festival, Holland
__ Aug. 20 (Monday) LifeWriters -- A group for people interested in writing the stories of their own lives or those of family and friends, 10 a.m., Shiloh Museum of Ozark History in downtown Springdale.
__ Aug. 21 (Tuesday) Booked for Lunch -- "Dog Medicine" by Julie Barton, noon, Springdale Public Library.
__ Aug. 26 (Sunday) Dickson Street Vault -- A pole vault competition benefiting Soldier On Service Dogs, 9 a.m.-9 p.m., Dickson Street in Fayetteville.


Celebrating Mid-Autumn Festival, Troy
Harbour Grand Kowloon, Hong Kong is celebr

COMPANION (AC) IN THE GENERAL DIVISION OF THE ORDER OF AUSTRALIA Professor Jeffrey Victor ROSENFELD AM,Caulfield North Vic 3161 For eminent service to medicine, particularly to the discipline of neurosurgery, as an academic and clinician, to medical research and professional organisations, and to the health and welfare of current and former defence force members.
OFFICER (AO) IN THE GENERAL DIVISION OF THE ORDER OF AUSTRALIA   David Lawrence BARDAS, South Yarra Vic 3141 For distinguished service to the retail fashion industry, to the Jewish community, and through philanthropic contributions to Indigenous, education, cultural, social welfare, visual arts, and to medical organisations.


Win Two Festival, Vancouver
The Dallas International Film Festival is one of best annual events in Dallas for good reason.
Every year, the Dallas Film Society showcases hundreds of soon-to-be or hoping-to-be released films for your viewing pleasure.
But which ones are worth your time?
Vote daily for the 

It’s a global community architects, designers and engineers that share their ideas and work on the designs together.
Not many people would have expected structural engineering project to become open-source.
The global network has become an important tool for exhorting power, in politics and business, for good and bad.
If you’re looking to start one, your motivation can be to make as much money as you can, but it could well be to make a very good living while helping others.


2
Southampton International Film Festival, Southampton
Ms Spurling told Bernews, “It is a huge vote of confidence to be nominated alongside films made with bigger budgets, and stars, and shows that our cast really raised the bar in our small film, which is not just a plot-driven thriller, it has several delightfully ambiguous characters, so our actors had to play their parts on several layers.” Maternal Secrets has been racking up awards and nominations since its launch including a Special Mention for Directing, t

Look for the current retrospective on the work of Japanese artist Takashi Murakami at Museum of Contemporary Art in Chicago entitled “The Octopus Eats Its Own Leg” to make its West Coast debut here in the winter of 2018.Vancouver Art Gallery is at 750 Hornby St. in Vancouver, BC Canada.
The Asia Society Museum in New York presents the following – On view through Jan. 21, 2018 is “After Darkness – Southeast Asian Art in The Wake of History.” Includes artists from Indonesia, Myanmar,and Vietnam.


22
67
Annual Great Pumpkin Beer Festival, Seattle
DragonFest This annual festival celebrates a variety of Asian cultures with two days of performances, food, and activities.
Bumbershoot 2018 Bumbershoot, Seattle's biggest music, comedy, and arts festival, will take over Seattle Center for Labor Day Weekend 2018 for the 48th year.
This year's lineup will most likely be bursting with local and national music stars, in addition to a stacked comedy circuit and related art events.


Light and Music 

After students watched the movie, they participated in a discussion with the filmmaker/producer, a former homeless youth who was featured in the film, and the president/founder of WIT, a San Diego-based organization that works with teens to empower them to become social entrepreneurs and leaders.
It’s thrilling to be part of The San Diego International Film Festival and support the work of the San Diego Film Foundation.” “We’re using film as a way to bring people together, to explore social issues and develop empathy,” says San Diego Film Festival CEO and artistic director Tonya Mantooth.


4
Adventure Film Festival, Boulder
Outside TV and Adventure Film Festival have partnered on a cool contest for their Campfire platform that will send one deserving winner to the Adventure Film Festival in Boulder, Colorado.
To enter the contest you first must register and create a Campfire channel on Outside TV.
To win the contest you, have to share your channel on social media sites and ask your fr

“The people and comradery are just amazing,” Butcher Salt food truck owner Jean Hutar said.
“Butcher Salt sliders have a little rosemary sage, thyme, margarine and cheese, bacon and pickle,” Hutar said.
MidNord Empanadas owner Phil Gaffney is no newcomer, but said the festival really shows off the Twin Cities food scene.
“The food truck revolution has really taken hold in the Minneapolis area,” Gaffney said.
If you are interested in attending, the Uptown Food Truck Festival is Sunday.


Circuit Festival, Miami
Like Bestival, it feels like a small Glastonbury, with each major stage complimented by a town district with its own theme (Wild West, Old Town, steam-punk, etc.) Although the 2017 edition was marred by poor organisation and enormous entry queues initially, acts ranging from The Specials and M.I.A. to Frank Turner made it a successful year.


9
Astroworld Festival, Houston
Travis Scott is floating somewhere in outer space after the release of his latest studio album Astroworld.
I

In [15]:
allText['Summary'] = summaries

In [16]:
from gensim.summarization.summarizer import summarize
summaries = []
for event, city in zip(allTextPr['Event Name'], allTextPr['City']):
    eventNews = allNewsPr[allNewsPr['Event Name']==event]
    count = 0
    countInd = 0
    for index, news in enumerate(eventNews['Article Text']):
        cityCount = news.count(city)
        eventCount = news.count(event)
        if cityCount*eventCount > count:
            count = cityCount*eventCount
            countInd = index
            print(count)
    print(event + ", " + city)
    try:
        summary = summarize(eventNews.iloc[countInd]['Article Text'], word_count = 80)
        summaries.append(summary.replace("\n", " "))
    except:
        summaries.append("")
        print("Error Summerizing!")
    print(summary)
    #print(eventNews.iloc[countInd]['Article Text'])
    print("\n")

3
Corona Sunsets Festival, Durban
This week’s BIG move:Vodacom Red partners with David Tlale With the Vodacom Durban July representing the nexus of South African fashion, society and horse-racing, luxury designer David Tlale and Vodacom Red have partnered for the first time in a unique experience, culminating in a limited collection, RED by David Tlale, at this year’s event.
“Vodacom Red is for the individual who expects more and we couldn’t think of anyone better than David Tlale to collaborate with in presenting an exceptional showcase of South African design at the Vodacom Durban July.


Peñafrancia Festival, Kasama
Travel time by land can take about 10 hours, which is why Bicol Isarog Transport System's newest sleeper bus that goes from Cubao to Naga City and vice versa is a welcome addition for transport choices to the province.
(L) comfort room; (R) USB charger Naga City annually hosts the grand Peñafrancia Festival, which is attended by hundreds of thousands of faithful devotees

Real Ruby Chocolate Experience, sanchurro.com SPRING COCKTAIL Saké Restaurant and Bar is preparing for cherry blossom season with beautiful floral installations on display in its bar at The Rocks throughout September.
2018 Lavazza Italian Film Festival, italianfilmfestival.com.au BERNADETTE ROBINSON IN CONCERT Australian singer and actor Bernadette Robinson is bringing a specially tailored program to Sydney.
It’s on at the Sydney Conservatorium of Music tonight, tickets $20-$50.


80
Drink Festival, Liverpool
High profile Merseyside chefs Ellis Barrie, Gary Usher and Anton Piotrowski are set to join Raymond Blanc OBE for a celebration of the city’s growing food and drink scene.
“I’m really looking forward to joining such a brilliant line-up of talented chefs to celebrate the city’s food offering, which just keeps on getting better.” Isabella Kantor, Liverpool Food and Drink Festival Manager, said: “Ellis Barrie, Gary Usher and Anton Piotrowski are all doing fantastic work for the regio

As you may know, this month we’ve been doing a focus on crowdfunding for filmmakers, and in a moment of synchronicity the IndieGoGo campaign for the 2018 Raindance Film Festival has begun!
So, why should you dig into your wallet and contribute to the crowdfunding campaign for this year’s Raindance Film Festival?
That’s quite an undertaking financially, but it’s one of the lengths Raindance wants to go to in order to be a more environmentally conscious festival and keep costs down for audiences at the same time.


STEM Festival, Cambridge
(WCBD) — The College of Charleston Foundation announced a $10,000 grant from the Dominion Energy Charitable Foundation, the non-profit arm of Dominion Energy, to fund Elementary Engineers and the Charleston STEM Festival.
Students are involved in hands-on experiments and challenges that reinforce STEM learning with the help of partners like Dominion Energy, the Grice Marine Lab and the Historic Charleston Foundation.
“We want to thank Dominion Energy f

The mission of Gelato Festival is to spread the culture of artisan Italian gelato, originally born in Florence, Italy.
Following the success of last year’s first Gelato Festival America, this year the Festival will tour eight cities in the US, with the winning gelato chefs from each city eventually competing in a grand finale and ultimately in the Gelato Festival World Master Finals in 2021.


During The Toronto International Film Festival, Temple
Actors Viggo Mortensen, left to right, director Peter Farrelly, Octavia Spencer and Mahershala Ali pose on the red carpet before the screening of "Green Book" during the Toronto International Film Festi...
Actor Mahershala Ali arrives ahead of the screening of "Green Book" during the Toronto International Film Festival in Toronto, on Tuesday, Sept.
Actor Mahershala Ali arrives ahead of the screening of "Green Book" during the Toronto International Film Festival in Toronto, on Tuesday, Sept.


1
11
20
Charlotte Film Festival, Charlotte
This an

The festival’s 5th year anniversary includes performances headlined by Deftones, Nine Inch Nails, Avenged Sevenfold and Godsmack.
The second day, Saturday September 29, is kicked off by Godsmack who released their first album in four years titled When Legends Rise.Five Finger Death Punch start their North American tour on June 14 in Canada.
The festival wraps up on Sunday, September 30 with headliners Nine Inch Nails who release their new album Halo 32 on June 22.


6
8
Iowa City Latino Festival, Iowa City
Manny Galvez, organizer of the annual Iowa City Latino Festival, which was set for this Saturday, announced Friday that they were postponing the event out of “respect for Mollie, her family and friends,” but also out of an abundance of caution after organizers became concerned with anti-immigrant vitriol on social media, The Hill reported.
Jon Wolseth, a member of Hispanics United for Perry, which organized the event, said they decided to follow in the Iowa City Latino Festival’s foo

15 Where: Santa Anita Park, Arcadia Information: vintagevibefest.com What you need to know: This festival started in Palm Springs last year as a way to celebrate all things retro, and this year’s event will have video games, photo booths, classic cars and more.
27-30 Where:Los Coyotes Indian Reservation, San Diego County Information: sacredvalleyfest.com What you need to know: This new festival from Insomniac will have two stages focusing on psychedelic dance music and will include yoga, wellness seminars and more.


20
Uranium Film Festival, Berlin
Another atomic bomb explodes, and the narrator asks, “What happened when we awoke the dragon?” The final frames announce the most recent International Uranium Film Festival, which took place in Berlin in autumn, 2017.
The International Uranium Film Festival is dedicated to all films about nuclear power and its associated risks of radioactivity.
The Uranium Film Festival Award is created from waste material directly from the the streets of S

Food Festival, Amritsar
“With over 100 nationalities living in Cayman, the Batabano Food Festival is testament to the spirit of Cayman’s history and welcoming heart.” Held on the same day as the Batabano Street Parade on May 5th – the first Saturday in May each year – the rebranded Batabano Food Festival Presented by Saxon, is the ultimate carnival street food extravaganza held on Cardinal Avenue in downtown George Town.


2
5
H Street Festival, Florence
“When you go outside today and look up, those clouds are a direct result of the hurricane,” said Storm Team4 Meteorologist Chuck Bell.
Some forecast models see Florence passing west of the Appalachians, consistent with earlier forecasts that it would dump heavy rain on southwestern Virginia.
(Some parts of this area could get up to 3–4 inches, too, Bell said.) For an already-water-logged region, that could mean more flooding and downed trees.
The good news: Ricketts said the weather will be fine when the Redskins host the Indianapolis 

RHPS EVENTS: A Celebration of Slashers- (April 28th, 9am-6pm) Celebrate slasher films in recognition of Halloween's 40th anniversary at the Pop Culture Conference at DePaul with horror-themed panels (see link for the list of speakers!) and special keynote speaker, director Rachel Talalay (Freddy's Dead: The Final Nightmare, Tank Girl, Riverdale, Doctor Who) and Dr. Wickham Clayton, lecturer in Film History and Theory, University for the Creative Arts, Farnham, UK.
Films will be screened at two locations: The New 400 Theatres (7101 N.


6
Fall Festival, Columbus
3 — Hancock County Fair, Findlay Aug. 31 — bison ‘76, Rally in the Square, Town Square, Lima Aug. 31 to Sept.
7 to 9 — Grand Lake St. Marys Fall Festival Sept.
14 to 16 — Lake Loramie Harvest Festival, Minster Sept.
21 to 23 — Flat Rock Creek Fall Festival, Paulding Sept.
21 to 23 — Fall Festival, Indian Lake Sept.


West Coast Festival, Tucson
Skye Wallace is on a quest to compile stark elements of Canadian landscape and histor

The Red Bull Music Festival is expanding to Chicago, and its Windy City debut will take place throughout November with, as the announcement says, a “month-long slate of expertly curated original programming spanning the city’s diverse music genres.” Highlights include the kickoff event on Nov. 3, where Kanye West’s G.O.O.D. Music will take over Wintrust Arena for an ensemble performance that features of Pusha T, Teyana Taylor, Valee, Sheck Wes, Desiigner, 070 Shake and special guest Nas. While West is not listed on the bill, it’s hard to imagine him not appearing.


Diwali Festival, Surprise
Applications are now open for stallholders for the 2018 Auckland Diwali Festival, which takes place on Saturday, 20 and Sunday, 21 October – Labour Weekend – in Aotea Square and Queen St. Auckland Diwali Festival is one of the region’s most popular cultural festivals, with 55,000 people attending the 2017 festival.
The festival is delivered by Auckland Tourism, Events and Economic Development (ATEE

In [17]:
allTextPr['Summary'] = summaries

In [18]:
allTextCom = pd.concat([allText, allTextPr])
allTextCom.drop_duplicates(subset=['Event Name', 'City'], inplace=True, keep='last')

In [20]:
allTextCom.to_pickle("Data/allText_Summarized_Sep27.pkl")


In [19]:
allTextCom.to_csv("Data/allText_Summarized_Sep27.csv")


## Do some pythonic and manual cleaning

In [21]:
for index, city in enumerate(allTextCom['City']):
    if city=="York":
        allTextCom.iloc[index]['City'] = "New York"
    if city=="Angeles":
        allTextCom.iloc[index]['City'] = "Los Angeles"
    if city=="Orléans":
        allTextCom.iloc[index]['City'] = "New Orleans"

In [22]:
allTextCom.to_csv("Data/allText_Summarized_Sep27.csv")

In [23]:
allText_final = pd.read_csv("Data/allText_Summarized_Sep27_edited.csv")

In [24]:
allText_final.to_pickle("Data/allText_final_Sep27.pkl")

In [28]:
allEvents.to_pickle("Data/allEventsSep28.pkl")

In [25]:
allText_final

,City,Event Name,Text,Summary
0,Bethlehem,Peanut Butter Festival,National Peanut Butter and Jelly Day is celebr...,Peanut Butter and Jelly sandwiches were though...
1,Durban,Lesbian Film Festival,A lesbian film festival is returning to Paduca...,A lesbian film festival is returning to Paduca...
2,Springs,Blue Springs Fall Fun Festival,"Burgers are a classic KC menu item. ""As a stea...",Cronin’s Bar & Grill in Lenexa — basic burger....
3,Bintulu,Borneo International Kite Festival,Click to share on WhatsApp (Opens in new windo...,Click to share on WhatsApp (Opens in new windo...
4,Los Angeles,LA Film Festival,Fresh off its ingestion of lip syncing app Mus...,Film Independent moved the festival from downt...
5,Kazan,Kazan International Festival,"International-award winning film Halda, direct...","International-award winning film Halda, direct..."
6,Bangkok,Irish Film Festival,Actor Colin Farrell has become a patron for th...,Actor Colin Farrell has become a patron for th...
7,Beijing,Mid-Autumn Festival,Millions of lights greet the Mid-Autumn Festiv...,The annual Mid-Autumn Festival will be here so...
8,San Francisco,Annual Iranian Film Festival,Cinephiles in Washington have no shortage of m...,Although they're best known for big-picture fe...
9,Sydney,Palestinian Film Festival,EVEREST CARNIVAL The Everest Carnival is comin...,EVEREST CARNIVAL The Everest Carnival is comin...
